# Data collection

In this notebook, I'll use the **GitHub API** to extract various information from vuejs profile (https://github.com/vuejs/vue) such as commit activity.

## Import libraries and defining constants

I'll import various libraries needed for fetching the data.

In [92]:
import json
import requests
import numpy as np
import pandas as pd
import base64
from github import Github
from pprint import pprint
import requests
from requests.auth import HTTPBasicAuth

In [158]:
import datetime
username = input("Enter the github username:")
request = requests.get('https://api.github.com/repos/vuejs/vue/stats/commit_activity')
#value = datetime.datetime.fromtimestamp(week)
#print(f"{value:%Y-%m-%d %H:%M:%S}")
data = request.json()
data

Enter the github username:vuejs


[{'total': 1, 'week': 1565481600, 'days': [0, 1, 0, 0, 0, 0, 0]},
 {'total': 1, 'week': 1566086400, 'days': [0, 0, 0, 0, 0, 0, 1]},
 {'total': 0, 'week': 1566691200, 'days': [0, 0, 0, 0, 0, 0, 0]},
 {'total': 1, 'week': 1567296000, 'days': [0, 0, 0, 1, 0, 0, 0]},
 {'total': 0, 'week': 1567900800, 'days': [0, 0, 0, 0, 0, 0, 0]},
 {'total': 2, 'week': 1568505600, 'days': [0, 0, 0, 0, 2, 0, 0]},
 {'total': 3, 'week': 1569110400, 'days': [0, 1, 1, 1, 0, 0, 0]},
 {'total': 1, 'week': 1569715200, 'days': [0, 0, 1, 0, 0, 0, 0]},
 {'total': 6, 'week': 1570320000, 'days': [4, 1, 1, 0, 0, 0, 0]},
 {'total': 0, 'week': 1570924800, 'days': [0, 0, 0, 0, 0, 0, 0]},
 {'total': 0, 'week': 1571529600, 'days': [0, 0, 0, 0, 0, 0, 0]},
 {'total': 11, 'week': 1572134400, 'days': [0, 10, 0, 1, 0, 0, 0]},
 {'total': 0, 'week': 1572739200, 'days': [0, 0, 0, 0, 0, 0, 0]},
 {'total': 3, 'week': 1573344000, 'days': [0, 0, 3, 0, 0, 0, 0]},
 {'total': 1, 'week': 1573948800, 'days': [0, 0, 1, 0, 0, 0, 0]},
 {'total

## So this data is not in readable format. Like total signifies the total number of commits, week represents the UNIX timestamp, days represents array a group of commits per day, starting on Sunday.

In [159]:
data[0]

{'total': 1, 'week': 1565481600, 'days': [0, 1, 0, 0, 0, 0, 0]}

In [165]:
repos_df = pd.DataFrame(data, columns = ['total', 'week','days'])
repos_df.head(10)

,total,week,days
0,1,1565481600,"[0, 1, 0, 0, 0, 0, 0]"
1,1,1566086400,"[0, 0, 0, 0, 0, 0, 1]"
2,0,1566691200,"[0, 0, 0, 0, 0, 0, 0]"
3,1,1567296000,"[0, 0, 0, 1, 0, 0, 0]"
4,0,1567900800,"[0, 0, 0, 0, 0, 0, 0]"
5,2,1568505600,"[0, 0, 0, 0, 2, 0, 0]"
6,3,1569110400,"[0, 1, 1, 1, 0, 0, 0]"
7,1,1569715200,"[0, 0, 1, 0, 0, 0, 0]"
8,6,1570320000,"[4, 1, 1, 0, 0, 0, 0]"
9,0,1570924800,"[0, 0, 0, 0, 0, 0, 0]"


## I need to convert UNIX timestamp to readable dateformat

In [166]:
repos_df['date'] = pd.to_datetime(repos_df['week'],unit='s')


repos_df.head(100)

,total,week,days,date
0,1,1565481600,"[0, 1, 0, 0, 0, 0, 0]",2019-08-11
1,1,1566086400,"[0, 0, 0, 0, 0, 0, 1]",2019-08-18
2,0,1566691200,"[0, 0, 0, 0, 0, 0, 0]",2019-08-25
3,1,1567296000,"[0, 0, 0, 1, 0, 0, 0]",2019-09-01
4,0,1567900800,"[0, 0, 0, 0, 0, 0, 0]",2019-09-08
5,2,1568505600,"[0, 0, 0, 0, 2, 0, 0]",2019-09-15
6,3,1569110400,"[0, 1, 1, 1, 0, 0, 0]",2019-09-22
7,1,1569715200,"[0, 0, 1, 0, 0, 0, 0]",2019-09-29
8,6,1570320000,"[4, 1, 1, 0, 0, 0, 0]",2019-10-06
9,0,1570924800,"[0, 0, 0, 0, 0, 0, 0]",2019-10-13


## Over here I spilted the date column into year and month

In [167]:
repos_df['year'] = repos_df['date'].dt.strftime('%Y')
repos_df['month'] = repos_df['date'].dt.month
repos_df.head(10)

,total,week,days,date,year,month
0,1,1565481600,"[0, 1, 0, 0, 0, 0, 0]",2019-08-11,2019,8
1,1,1566086400,"[0, 0, 0, 0, 0, 0, 1]",2019-08-18,2019,8
2,0,1566691200,"[0, 0, 0, 0, 0, 0, 0]",2019-08-25,2019,8
3,1,1567296000,"[0, 0, 0, 1, 0, 0, 0]",2019-09-01,2019,9
4,0,1567900800,"[0, 0, 0, 0, 0, 0, 0]",2019-09-08,2019,9
5,2,1568505600,"[0, 0, 0, 0, 2, 0, 0]",2019-09-15,2019,9
6,3,1569110400,"[0, 1, 1, 1, 0, 0, 0]",2019-09-22,2019,9
7,1,1569715200,"[0, 0, 1, 0, 0, 0, 0]",2019-09-29,2019,9
8,6,1570320000,"[4, 1, 1, 0, 0, 0, 0]",2019-10-06,2019,10
9,0,1570924800,"[0, 0, 0, 0, 0, 0, 0]",2019-10-13,2019,10


## To get stats for 2020 in particular

In [204]:
repos_df.rename(columns={'total': 'number_of_commits'}, inplace=True)
commit_df = repos_df.loc[repos_df['year'] == '2020']
commit_df

,number_of_commits,week,days,date,year,month
21,2,1578182400,"[0, 2, 0, 0, 0, 0, 0]",2020-01-05,2020,1
22,0,1578787200,"[0, 0, 0, 0, 0, 0, 0]",2020-01-12,2020,1
23,0,1579392000,"[0, 0, 0, 0, 0, 0, 0]",2020-01-19,2020,1
24,1,1579996800,"[1, 0, 0, 0, 0, 0, 0]",2020-01-26,2020,1
25,3,1580601600,"[0, 0, 0, 2, 0, 1, 0]",2020-02-02,2020,2
26,5,1581206400,"[0, 0, 5, 0, 0, 0, 0]",2020-02-09,2020,2
27,0,1581811200,"[0, 0, 0, 0, 0, 0, 0]",2020-02-16,2020,2
28,1,1582416000,"[0, 0, 0, 0, 1, 0, 0]",2020-02-23,2020,2
29,1,1583020800,"[0, 0, 0, 0, 0, 1, 0]",2020-03-01,2020,3
30,4,1583625600,"[0, 3, 1, 0, 0, 0, 0]",2020-03-08,2020,3


In [209]:
sum_df = commit_df.groupby(['year','month']).agg({'number_of_commits': 'sum'})
sum_df


number_of_commits
year month                   
2020 1                      3
     2                      9
     3                      7
     4                      5
     5                      4
     6                      1
     7                      1
     8                      3

In [215]:
Total = sum(sum_df['number_of_commits'])
print ("Total commits till date for 2020:", Total)

Total commits till date for 2020: 33


## So in the ablove cell you can see the stats of commits for 2020 as per the month and the Total